In [26]:
from ucimlrepo import fetch_ucirepo 
import polars as pl

In [27]:

# fetch dataset 
student_performance = fetch_ucirepo(id=320) 
  
# data (as pandas dataframes) 
X = pl.from_pandas(student_performance.data.features)
y = pl.from_pandas(student_performance.data.targets)

In [28]:
X

school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
str,str,i64,str,str,str,i64,i64,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64
"""GP""","""F""",18,"""U""","""GT3""","""A""",4,4,"""at_home""","""teacher""","""course""","""mother""",2,2,0,"""yes""","""no""","""no""","""no""","""yes""","""yes""","""no""","""no""",4,3,4,1,1,3,4
"""GP""","""F""",17,"""U""","""GT3""","""T""",1,1,"""at_home""","""other""","""course""","""father""",1,2,0,"""no""","""yes""","""no""","""no""","""no""","""yes""","""yes""","""no""",5,3,3,1,1,3,2
"""GP""","""F""",15,"""U""","""LE3""","""T""",1,1,"""at_home""","""other""","""other""","""mother""",1,2,0,"""yes""","""no""","""no""","""no""","""yes""","""yes""","""yes""","""no""",4,3,2,2,3,3,6
"""GP""","""F""",15,"""U""","""GT3""","""T""",4,2,"""health""","""services""","""home""","""mother""",1,3,0,"""no""","""yes""","""no""","""yes""","""yes""","""yes""","""yes""","""yes""",3,2,2,1,1,5,0
"""GP""","""F""",16,"""U""","""GT3""","""T""",3,3,"""other""","""other""","""home""","""father""",1,2,0,"""no""","""yes""","""no""","""no""","""yes""","""yes""","""no""","""no""",4,3,2,1,2,5,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""MS""","""F""",19,"""R""","""GT3""","""T""",2,3,"""services""","""other""","""course""","""mother""",1,3,1,"""no""","""no""","""no""","""yes""","""no""","""yes""","""yes""","""no""",5,4,2,1,2,5,4
"""MS""","""F""",18,"""U""","""LE3""","""T""",3,1,"""teacher""","""services""","""course""","""mother""",1,2,0,"""no""","""yes""","""no""","""no""","""yes""","""yes""","""yes""","""no""",4,3,4,1,1,1,4
"""MS""","""F""",18,"""U""","""GT3""","""T""",1,1,"""other""","""other""","""course""","""mother""",2,2,0,"""no""","""no""","""no""","""yes""","""yes""","""yes""","""no""","""no""",1,1,1,1,1,5,6


In [29]:
y

G1,G2,G3
i64,i64,i64
0,11,11
9,11,11
12,13,12
14,14,14
11,13,13
…,…,…
10,11,10
15,15,16
11,12,9


- Consideraremos un subconjunto de predicrtores, para simplificar el problema (le pedí a chatgpt que me escogiera ese subconjunto en base a el potencial explicativo de los predictores sobre la respuesta G3).

- Hay que categorizar G3 ya que tal cual vienen es una respuesta cuantitativa, necesitamos tenerla por categorias del tipo suspenso, bien, notable y aprobado, por ejemplo. Primero llevar G3 a la escala tipica de notas 0-10, ya que está en 0-20, para ello dividirla entre 2, luego aplicar categorización por intervalos (ej: [0,5): suspenos, [5,6): suficiente, etc)

In [30]:
y = y.with_columns(pl.col('G3') / 2)
y

G1,G2,G3
i64,i64,f64
0,11,5.5
9,11,5.5
12,13,6.0
14,14,7.0
11,13,6.5
…,…,…
10,11,5.0
15,15,8.0
11,12,4.5


In [31]:
y = y.with_columns(pl.col('G3').cut(
    breaks=[5, 6, 7, 9], 
    labels = ["suspenso", "suficiente", "bien", "notable", "sobresaliente"],
    left_closed=True
).alias('G3_cat')
)

y

G1,G2,G3,G3_cat
i64,i64,f64,cat
0,11,5.5,"""suficiente"""
9,11,5.5,"""suficiente"""
12,13,6.0,"""bien"""
14,14,7.0,"""notable"""
11,13,6.5,"""bien"""
…,…,…,…
10,11,5.0,"""suficiente"""
15,15,8.0,"""notable"""
11,12,4.5,"""suspenso"""


In [32]:
y.filter(pl.col('G3') >= 9)

G1,G2,G3,G3_cat
i64,i64,f64,cat
18,17,9.0,"""sobresaliente"""
17,17,9.0,"""sobresaliente"""
16,17,9.0,"""sobresaliente"""
16,17,9.0,"""sobresaliente"""
17,17,9.0,"""sobresaliente"""
…,…,…,…
18,18,9.0,"""sobresaliente"""
17,18,9.0,"""sobresaliente"""
18,18,9.0,"""sobresaliente"""


In [33]:
selected_response = 'G3_cat'
y = y[selected_response]

In [34]:
selected_predictors = ["failures", "absences", "studytime", "schoolsup", "famsup", "Medu", "Fedu", "famrel", "goout", "Walc", "health"]

X = X[selected_predictors]

In [35]:
X

failures,absences,studytime,schoolsup,famsup,Medu,Fedu,famrel,goout,Walc,health
i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64
0,4,2,"""yes""","""no""",4,4,4,4,1,3
0,2,2,"""no""","""yes""",1,1,5,3,1,3
0,6,2,"""yes""","""no""",1,1,4,2,3,3
0,0,3,"""no""","""yes""",4,2,3,2,1,5
0,0,2,"""no""","""yes""",3,3,4,2,2,5
…,…,…,…,…,…,…,…,…,…,…
1,4,3,"""no""","""no""",2,3,5,2,2,5
0,4,2,"""no""","""yes""",3,1,4,4,1,1
0,6,2,"""no""","""no""",1,1,1,1,1,5


In [36]:
X['schoolsup'].dtype

String

In [37]:
cols_to_encode = [col for col in X.columns if X[col].dtype == pl.String]
cols_to_encode

['schoolsup', 'famsup']

In [ ]:
encoding = {}
for col in cols_to_encode:
    unique_values_sorted = sorted(X[col].unique().to_list())
    new_values = list(range(0, len(unique_values_sorted)))
    encoding[col] = dict(zip(unique_values_sorted, new_values))
    
for col in cols_to_encode: 
    X = X.with_columns(pl.col(col).replace_strict(encoding[col]).alias(col))

In [41]:
X

failures,absences,studytime,schoolsup,famsup,Medu,Fedu,famrel,goout,Walc,health
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,4,2,1,0,4,4,4,4,1,3
0,2,2,0,1,1,1,5,3,1,3
0,6,2,1,0,1,1,4,2,3,3
0,0,3,0,1,4,2,3,2,1,5
0,0,2,0,1,3,3,4,2,2,5
…,…,…,…,…,…,…,…,…,…,…
1,4,3,0,0,2,3,5,2,2,5
0,4,2,0,1,3,1,4,4,1,1
0,6,2,0,0,1,1,1,1,1,5


In [ ]:
# Sort variables according to data type
len_unique_values = {}
for col in X.columns:
    len_unique_values[col] = len(X[col].unique())
quant_predictors = ['absences', 'failures']
cat_predictors = [col for col in X.columns if col not in quant_predictors]
binary_predictors = [col for col in cat_predictors if len_unique_values[col] == 2]
multiclass_predictors = [col for col in cat_predictors if col not in binary_predictors]

In [52]:
binary_predictors

['schoolsup', 'famsup']

In [53]:
multiclass_predictors

['studytime', 'Medu', 'Fedu', 'famrel', 'goout', 'Walc', 'health']

| Variable   | Descripción                                                                 | Tipo                  | Justificación                                                                                      | Valores únicos |
|------------|-----------------------------------------------------------------------------|-----------------------|---------------------------------------------------------------------------------------------------|----------------|
| failures   | Número de asignaturas reprobadas previamente.                               | Numérica (entera)     | Refleja historial académico previo; alto valor suele asociarse a menor rendimiento futuro.         | 0, 1, 2, 3    |
| absences   | Número de ausencias a clase.                                                | Numérica (entera)     | La asistencia está directamente ligada al aprendizaje y a las calificaciones finales.              | 0 – 75        |
| studytime  | Tiempo de estudio semanal (categorías: 1–4).                                | Categórica ordinal    | Mide hábitos de estudio; más tiempo suele correlacionar con mejor rendimiento.                     | 1, 2, 3, 4    |
| schoolsup  | Apoyo educativo extra proporcionado por el colegio.                         | Binaria               | Representa recursos de refuerzo académico institucional.                                           | yes, no       |
| famsup     | Apoyo familiar en los estudios.                                             | Binaria               | El respaldo familiar influye en motivación y continuidad escolar.                                 | yes, no       |
| Medu       | Nivel educativo de la madre (0–4).                                          | Categórica ordinal    | La educación parental es un fuerte predictor del rendimiento académico de los hijos.              | 0, 1, 2, 3, 4|
| Fedu       | Nivel educativo del padre (0–4).                                            | Categórica ordinal    | Similar a `Medu`, añade contexto sociofamiliar relevante.                                          | 0, 1, 2, 3, 4|
| famrel     | Calidad de las relaciones familiares (1–5).                                 | Categórica ordinal    | Un entorno familiar estable favorece la concentración y el rendimiento escolar.                   | 1, 2, 3, 4, 5|
| goout      | Frecuencia de salidas con amigos (1–5).                                     | Categórica ordinal    | Exceso de ocio social puede competir con tiempo de estudio y afectar negativamente al rendimiento.| 1, 2, 3, 4, 5|
| Walc       | Consumo de alcohol en fines de semana (1–5).                                | Categórica ordinal    | Asociado a distracciones, menor rendimiento y problemas de salud que repercuten en los estudios.  | 1, 2, 3, 4, 5|
| health     | Estado de salud autoinformado (1–5).                                        | Categórica ordinal    | La salud condiciona la asistencia, la concentración y el rendimiento académico.                   | 1, 2, 3, 4, 5|
| G3         | Nota final del curso (0–20).                                                | Numérica (entera) (categorica ordinal, tras procesado)    | Variable objetivo a predecir, indicador del rendimiento académico global.                         | 0 – 20   ( ["suspenso", "suficiente", "bien", "notable", "sobresaliente"] tras transformarla a la escala 0 - 10 y categorizarla)    |
